<a href="https://colab.research.google.com/github/fladdict/stable-diffusion/blob/main/Stable_Diffusion_Helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion 体験版

画像生成AI [StableDiffusion](https://github.com/CompVis/stable-diffusion)をGUIで使えるノートブックです。

## 使い方

* このページ上部のメニューで、「ランタイム > ランタイムのタイプを変更」からGPUを有効化
* [HuggingFace](https://huggingface.co/)でアカウントを作成
* [StableDiffusionのモデルページ](https://huggingface.co/CompVis/stable-diffusion-v-1-4-original)で、「利用規約」に合意する。
* モデルファイル [sd-v1-4.ckpt](https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt) をダウンロード
* モデルファイルを Google Drive等にアップロード
* 下のセル 「1-1. Google Driveとの接続」を実行
* 下のセル　「1-2. のフォーム」に、Google Driveにアップしたモデルのパスをセット
* このページ上部のメニューで、「ランタイム > 全てのセルを実行」を選択
* 一番したのほうにGUIが出現する。（近くのURLで別窓でも開ける）。

# 重要

「[Stable Diffusionの利用ライセンス](https://huggingface.co/spaces/CompVis/stable-diffusion-license)」を遵守してご利用ください。

----

# 利用前の注意
画像生成AIは、インターネットそのものの縮図です。あらゆるものを生成するので、生成者は自分の生成物に責任をもつ必要があります。

多くの場合、問題ある画像は「生成者が意図的に指示」をすることで生成されます。以下のようなことを心がけましょう。


* ポルノを含む、性的な画像を生成しない（海外基準で罰せられる可能性があります）。
* 攻撃的な画像、差別的な画像、人を不快にする目的の画像を生成しない。
* 政治的な主張に用いない。
* 各種の文化バイアスがかかる場合があります。生成者が適宜バランスを調整をする（例、「結婚式」の画像は欧米式で異性愛の画像になりやすい。医者の画像は白人男性になりやすい）。
* 他者の権利を侵害しない（孫悟空やダースベイダーなどを意図的に作らない）
* 実材の人物、事件、イベントの画像（フェイクニュース含む）を作成しない
* 他人の著作物をベース画像として用いない
* 現役の作家の画風を単独指名で入力しない（個人的に推奨のマナーです）

----

# お願い
AIによる画像生成、仕事がなくなるといった文脈で煽る方向の流れは、望むものではありません。
むしろ、みんなで「新しい創作」はどういうものか？アーティストはどうAIを使いこなしていけばいいのか？を模索していければお思います。 

活版印刷が著作権の概念を生み、写真が印象派や抽象芸術の扉を開いたように、新しいテクノロジーは、新しい表現をもたらします。今、必要なことは、みんなであらゆる方向から実験をして、新しい可能性の総当たり探索をすることだと思います。

そんな方向性で使ってもらえればと。


# 謝辞
構成コードは以下の方々のライブラリ、スニペット、コードを参考、あるいは依拠しています。
またnotebookは下記コード群のライセンスを継承します。

* 基本コードは、[StableDiffusion](https://github.com/CompVis/stable-diffusion) と [Diffusers](https://github.com/huggingface/diffusers)
* KLMSサンプリングに、[@RiversHaveWings](https://twitter.com/RiversHaveWings) 氏の [KLMS Sampling](https://github.com/crowsonkb/k-diffusion.git)より
* プロンプトのウェイト処理は、[@Lincoln Stein](https://github.com/lstein)氏のカスタム版[StableDiffusion](https://github.com/lstein/stable-diffusion)より

# 更新履歴

* セーフフィルタのリックおじさんを空白画像に差し替え
* プロンプトにMidJourney風ウェイト処理を追加。「dog::5 cat::3」などとできる。
* DiffuserベースだとImg2Imgが限定的だったので、Diffuserやめる。
* Gradioで最低限のGUIをつける

# Setup

In [97]:
#@markdown ## 1-1. Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
#@markdown ## 1-2. Google Driveにアップしたモデルのパスを設定
#@markdown 左メニューからGoogle Driveを掘り、アップロードしたckptファイルを選択し、右クリックから「パスをコピー」を行います。
#@markdown コピーしたパスを下のフォームにコピペしてください。

GDRIVE_MODEL_PATH = "/content/drive/MyDrive/stable-diffusion/sd-v1-4.ckpt" #@param {type:"string"}

In [99]:
#パスの環境変数
DRIVE_PATH = "/content/drive/MyDrive" #Driveのルート
DRIVE_SD_PATH = DRIVE_PATH + "/stable-diffusion"  #Drive内のStableDiffusion用のパス
DRIVE_SD_OUTPUT_PATH = DRIVE_SD_PATH + "/output" #画像を保存するパス

In [100]:
#GPUの確認
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-fea6c688-fd9f-08f2-2ae2-419a6d96af06)


In [ ]:
#必要ファイルのインストール

#GIT
!git clone https://github.com/CompVis/stable-diffusion  
%cd stable-diffusion/

!git clone https://github.com/CompVis/taming-transformers
!git clone https://github.com/openai/CLIP.git
!git clone https://github.com/crowsonkb/k-diffusion.git

#PIP
!pip install diffusers==0.2.4 
!pip install gradio
!pip install pytorch-lightning
!pip install torch-fidelity
!pip install numpy omegaconf einops kornia
!pip install albumentations transformers
!pip install ftfy jsonmerge resize-right torchdiffeq tqdm

#Pathを通す
import sys
sys.path.append(".")
sys.path.append("./CLIP")
sys.path.append('./taming-transformers')
sys.path.append('./k-diffusion')

In [ ]:
import argparse, gc, json, os, random, sys, time, glob, requests
import torch
import torch.nn as nn
import numpy as np
import PIL
from contextlib import contextmanager, nullcontext
from einops import rearrange, repeat
from IPython.display import display, clear_output
from itertools import islice
from omegaconf import OmegaConf
from PIL import Image
from pytorch_lightning import seed_everything
from torch.cuda.amp import autocast
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

from k_diffusion.sampling import sample_lms
from k_diffusion.external import CompVisDenoiser

from diffusers.pipelines.stable_diffusion.safety_checker import StableDiffusionSafetyChecker
from transformers import AutoFeatureExtractor


#新しいDenoiser
class CFGDenoiser(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.inner_model = model

    def forward(self, x, sigma, uncond, cond, cond_scale):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])
        uncond, cond = self.inner_model(x_in, sigma_in, cond=cond_in).chunk(2)
        return uncond + (cond - uncond) * cond_scale


#Config用クラス
class SDOption():
  def __init__(self):
    self.ckpt = GDRIVE_MODEL_PATH
    self.config = 'configs/stable-diffusion/v1-inference.yaml'
    self.ddim_eta = 0.0
    self.ddim_steps = 50
    self.fixed_code = True
    self.init_img = None
    self.n_iter = 1
    self.n_samples = 1
    self.outdir = DRIVE_SD_OUTPUT_PATH
    self.precision = 'full' # 'autocast'
    self.prompt = ""
    self.sampler = 'klms'
    self.scale = 7.5
    self.seed = 42
    self.strength = 0.5
    self.H = 512
    self.W = 512
    self.C = 4
    self.f = 8


class SDHelper():
  def __init__(self, config_path, model_path):
    config = OmegaConf.load(config_path)
    self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    self.model = self.load_model_from_config(config, model_path).to(self.device)
    self.safety_model_id = "CompVis/stable-diffusion-safety-checker"
    self.safety_feature_extractor = AutoFeatureExtractor.from_pretrained(self.safety_model_id)
    self.safety_checker = StableDiffusionSafetyChecker.from_pretrained(self.safety_model_id)

  def chunk(self, it, size):
      it = iter(it)
      return iter(lambda: tuple(islice(it, size)), ())


  def load_model_from_config(self, config, ckpt, verbose=False):
      print(f"Loading model from {ckpt}")
      pl_sd = torch.load(ckpt, map_location="cpu")
      if "global_step" in pl_sd:
          print(f"Global Step: {pl_sd['global_step']}")
      sd = pl_sd["state_dict"]
      model = instantiate_from_config(config.model)
      m, u = model.load_state_dict(sd, strict=False)
      if len(m) > 0 and verbose:
          print("missing keys:")
          print(m)
      if len(u) > 0 and verbose:
          print("unexpected keys:")
          print(u)

      model.cuda()
      model.eval()
      return model


  def load_img(self, path, w, h):
      if path.startswith('http://') or path.startswith('https://'):
          image = Image.open(requests.get(path, stream=True).raw).convert('RGB')
      else:
          if os.path.isdir(path):
              files = [file for file in os.listdir(path) if file.endswith('.png') or file .endswith('.jpg')]
              path = os.path.join(path, random.choice(files))
              print(f"Chose random init image {path}")
          image = Image.open(path).convert('RGB')
      image = image.resize((w, h), Image.LANCZOS)
      w, h = image.size
      w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
      image = image.resize((w, h), resample=PIL.Image.LANCZOS)
      image = np.array(image).astype(np.float32) / 255.0
      image = image[None].transpose(0, 3, 1, 2)
      image = torch.from_numpy(image)
      return 2.*image - 1.


  def numpy_to_pil(self, images):
    if images.ndim == 3:
        images = images[None, ...]
    images = (images * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images


  def load_replacement(self, x):
      try:
          hwc = x.shape
          #セーフフィルターのリック・ストレイは、日本では法律に触れる可能性があるので、違う画像に差し替えます。
          #y = Image.open("assets/rick.jpeg").convert("RGB").resize((hwc[1], hwc[0]))
          y = PIL.Image.new(mode="RGB", size=(hwc[1], hwc[0]))
          y = (np.array(y)/255.0).astype(x.dtype)
          assert y.shape == x.shape
          return y
      except Exception:
          return x


  def check_safety(self, x_image):
    safety_checker_input = self.safety_feature_extractor(self.numpy_to_pil(x_image), return_tensors="pt")
    x_checked_image, has_nsfw_concept = self.safety_checker(images=x_image, clip_input=safety_checker_input.pixel_values)
    assert x_checked_image.shape[0] == len(has_nsfw_concept)
    for i in range(len(has_nsfw_concept)):
        if has_nsfw_concept[i]:
            x_checked_image[i] = self.load_replacement(x_checked_image[i])
    return x_checked_image, has_nsfw_concept



  def generate(self, opt):
      global sample_idx
      seed_everything(opt.seed)

      #出力ディレクトリの作成
      os.makedirs(opt.outdir, exist_ok=True)

      if opt.sampler == 'plms':
          sampler = PLMSSampler(self.model)
      else:
          sampler = DDIMSampler(self.model)

      model_wrap = CompVisDenoiser(self.model)       
      batch_size = opt.n_samples
      prompt = opt.prompt
      assert prompt is not None
      data = [batch_size * [prompt]]
      init_latent = None

      if opt.init_img != None and opt.init_img != '':
          init_image = self.load_img(opt.init_img, opt.W, opt.H).to(self.device)
          init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
          init_latent = self.model.get_first_stage_encoding(self.model.encode_first_stage(init_image))  # move to latent space

      sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)

      t_enc = int(opt.strength * opt.ddim_steps)

      start_code = None
      if opt.fixed_code and init_latent == None:
          start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=self.device)

      precision_scope = autocast if opt.precision == "autocast" else nullcontext

      images = []
      with torch.no_grad():
          with precision_scope("cuda"):
              with self.model.ema_scope():
                  for n in range(opt.n_iter):
                      for prompts in data:
                          uc = None
                          if opt.scale != 1.0:
                              uc = self.model.get_learned_conditioning(batch_size * [""])
                          if isinstance(prompts, tuple):
                              prompts = list(prompts)

                          #プロンプトのウェイト処理
                          #全てのプロンプトに正規化したウェイトをかけて合算する
                          subprompts, weights = SDHelper.prompt_splitter(prompts[0])
                          if len(subprompts) > 1:
                            # i dont know if this is correct.. but it works
                            c = torch.zeros_like(uc)
                            # get total weight for normalizing
                            totalWeight = sum(weights)
                            # normalize each "sub prompt" and add it
                            for i in range(0,len(subprompts)):
                              weight = weights[i]
                              #if not skip_normalize:
                              # skip_normalizeがついてる意図が不明なので外す。
                              weight = weight / totalWeight
                              c = torch.add(c,self.model.get_learned_conditioning(subprompts[i]), alpha=weight)
                          else: # just standard 1 prompt
                            c = self.model.get_learned_conditioning(prompts)

                          #c = self.model.get_learned_conditioning(prompts)

                          if init_latent != None:
                              z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(self.device))
                              samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                                      unconditional_conditioning=uc,)
                          else:

                              if opt.sampler == 'klms':
                                  print("Using KLMS sampling")
                                  shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                                  sigmas = model_wrap.get_sigmas(opt.ddim_steps)
                                  model_wrap_cfg = CFGDenoiser(model_wrap)
                                  x = torch.randn([opt.n_samples, *shape], device=self.device) * sigmas[0]
                                  extra_args = {'cond': c, 'uncond': uc, 'cond_scale': opt.scale}
                                  samples = sample_lms(model_wrap_cfg, x, sigmas, extra_args=extra_args, disable=False)
                              else:
                                  shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                                  samples, _ = sampler.sample(S=opt.ddim_steps,
                                                                  conditioning=c,
                                                                  batch_size=opt.n_samples,
                                                                  shape=shape,
                                                                  verbose=False,
                                                                  unconditional_guidance_scale=opt.scale,
                                                                  unconditional_conditioning=uc,
                                                                  eta=opt.ddim_eta,
                                                                  x_T=start_code)

                          x_samples = self.model.decode_first_stage(samples)
                          x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
                          x_samples = x_samples.cpu().permute(0, 2, 3, 1).numpy()

                          #Safety Checker added
                          x_checked_image, has_nsfw_concept = self.check_safety(x_samples)
                          x_checked_image_torch = torch.from_numpy(x_checked_image).permute(0, 3, 1, 2)

                          for x_sample in x_checked_image_torch:
                              x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                              images.append(Image.fromarray(x_sample.astype(np.uint8)))
                              #filepath = os.path.join(opt.outdir, f"{batch_name}({batch_idx})_{sample_idx:04}.png")
                              #print(f"Saving to {filepath}")
                              #Image.fromarray(x_sample.astype(np.uint8)).save(filepath)
                              #sample_idx += 1
      return images

  #Prompt Splitter is based on Lincoln Stein's code
  #https://github.com/lstein/stable-diffusion/blob/main/ldm/simplet2i.py

  #MidJourney互換でプロンプトの重さを処理するコード
  #任意の文字列、 :: （スペース入るかも）（数字はいるかも）　（スペース入るかも）
  def prompt_splitter(text):
    """
    grabs all text up to the first occurrence of ':' 
    uses the grabbed text as a sub-prompt, and takes the value following ':' as weight
    if ':' has no value defined, defaults to 1.0
    repeats until no text remaining
    """
    remaining = len(text)
    prompts = []
    weights = []
    while remaining > 0:
        if "::" in text:
            idx = text.index("::") # first occurrence from start
            # grab up to index as sub-prompt
            prompt = text[:idx]
            remaining -= idx
            # remove from main text
            text = text[idx+2:]
            # find value for weight 
            if " " in text:
                idx = text.index(" ") # first occurence
            else: # no space, read to end
                idx = len(text)
            if idx != 0:
                try:
                    weight = float(text[:idx])
                except: # couldn't treat as float
                    print(f"Warning: '{text[:idx]}' is not a value, are you missing a space?")
                    weight = 1.0
            else: # no value found
                weight = 1.0
            # remove from main text
            remaining -= idx
            text = text[idx+1:]
            # append the sub-prompt and its weight
            prompts.append(prompt)
            weights.append(weight)
        else: # no : found
            if len(text) > 0: # there is still text though
                # take remainder as weight 1
                prompts.append(text)
                weights.append(1.0)
            remaining = 0
    print(prompts)
    print(weights)
    return prompts, weights


#SDHelperのインスタンス化
opt = SDOption()
sdh = SDHelper(opt.config, opt.ckpt)

In [ ]:
#GUIを起動

import random
import torch
import gradio as gr

import gc
gc.collect()
torch.cuda.empty_cache() 


def diffuse(init_image, prompt, width, height, guidance_scale, steps, init_strength, num, seed):
  result = list()

  opt.init_img = init_image
  opt.strength = 1- init_strength
  opt.prompt = prompt
  opt.W = width
  opt.H = height
  opt.scale = guidance_scale
  opt.ddim_steps = steps
  opt.n_iter = 1
  
  for index in range(int(num)):
    opt.seed = random.randint(0, 2**32) if seed == -1 else seed
    image = sdh.generate(opt)[0]
    display(image)
    result.append(image)
  return result


#GUI
css = "#gallery {width: 640px　!important; height: 640px　!important;} .output-image, .input-image, .image-preview {height: 512px !important}"
demo = gr.Interface(fn = diffuse, 
                    inputs = [gr.Image(label="Init Image(option)", tool="select", type="filepath"),
                              gr.Textbox(label="Prompt", value="Beautiful detailing landscape oil painting of river and forest in the style of realism, perfect composition, golden hour"), 
                              gr.Slider(label="Width", value=512, step=64, minimum=512, maximum=1024), 
                              gr.Slider(label="Height", value=512, step=64, minimum=512, maximum=1024),
                              gr.Slider(label="Cfg Scale", value=7.5, step=0.5, minimum=0, maximum=20),
                              gr.Slider(label="Steps", value=50, step=10, minimum=10, maximum=150),
                              gr.Slider(label="Init Image Strength", value=0.8, step=0.05, minimum=0, maximum=1), 
                              gr.Number(label="Num Images", value=1, minimum=0, maximum=9),
                              gr.Number(label="Seed", value=-1)],
                    outputs = gr.Gallery(elem_id="gallery").style(height="640px", container=True),
                    examples = [[None, "Beautiful detailing landscape oil painting of river and forest in the style of realism, perfect composition, golden hour", 512, 512, 7.5, 50, 0.8, 1, -1],
                                 [None, "High quality water color painting of cat in the style of picture book", 512, 512, 7.5, 50, 0.8, 1, -1],
                                 [None, "Stunning portrait painting of old man with glass in the style of impressionism, museum master collection", 512, 512, 7.5, 50, 0.8, 1, -1]],
                    css = css,
                    title = "Stable Diffusion Helper",
                    allow_flagging = "never",
                    show_error =True)

demo.launch(debug=True)